<a href="https://colab.research.google.com/github/shresht77/autograder-for-concept-maps/blob/main/autograder_cmap_no_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install rake-nltk
from rake_nltk import Rake

In [ ]:
!pip install git+https://github.com/LIAAD/yake

In [ ]:
import yake

In [ ]:
kw_extractor = yake.KeywordExtractor()
a= 'Sleuthkit.txt'
out=a.replace(".txt","")
out = out+".csv"
print(out)
with open(a, 'r') as file:
    data = file.read().replace('\n', ' ')

# Extraction given the text.
one = ""
keywords = kw_extractor.extract_keywords(data)
print(keywords)
for i in keywords:
  a=i[0]
  print (a)
  one = one+" "+a

print (one)

In [ ]:
r = Rake()
a= 'Sleuthkit.txt'
out=a.replace(".txt","")
out = out+".csv"
print(out)
with open(a, 'r') as file:
    data = file.read().replace('\n', ' ')

# Extraction given the text.
r.extract_keywords_from_text(data)
rank = r.get_ranked_phrases()
print(rank)
rank_res = " ".join(rank)
print(rank_res)

In [ ]:
import csv
with open(out, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows([rank_res])

# Wordnet implementation

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [ ]:
import re
import glob
import os
import csv
import shutil
import collections
from collections import defaultdict
from collections import Counter
from xml.dom import minidom

In [ ]:
fil1 = 'Sleuthkit.cxl'
xmldoc = minidom.parse(fil1)
out_fi = fil1.replace(".cxl","")
out_fil = "/content/drive/My Drive/Auto_waterloo/res/try/" + out_fi + "_link.csv"
out_fi = "/content/drive/My Drive/Auto_waterloo/res/try/" + out_fi + "_nodelink.csv"
conceptlist = xmldoc.getElementsByTagName('concept')
linklist = xmldoc.getElementsByTagName('linking-phrase')
conlist = xmldoc.getElementsByTagName('connection')

concept={}
link={}
connection={}
connection_id={}
connection_from={}
connection_to={}
final={}
final_f=collections.defaultdict(list)
no=[]

l_len=len(linklist)
c_len=len(conceptlist)
co_len=len(conlist)
link_master = []
label_master = []
final_connect = []

a=0
while a<l_len:
  link[linklist[a].attributes['id'].value]=linklist[a].attributes['label'].value
  link_master.append(linklist[a].attributes['label'].value)
  a = a+1
  link = { k:v.replace("\n"," ") for k, v in link.items()}
  link = { k:v.replace("\n"," ") for k, v in link.items()}
  link = { k:v.replace("  "," ") for k, v in link.items()}
  link_master = [item.replace("\n", " ") for item in link_master]
  link_master = [item.replace("\r", " ") for item in link_master]
  link_master = [item.replace("  ", " ") for item in link_master]

a=0
while a<c_len:
  concept[conceptlist[a].attributes['id'].value] = conceptlist[a].attributes['label'].value
  label_master.append(conceptlist[a].attributes['label'].value)
  a = a+1
  concept = { k:v.replace("\n"," ") for k, v in concept.items()}
  concept = { k:v.replace("\r"," ") for k, v in concept.items()}
  concept = { k:v.replace("  "," ") for k, v in concept.items()}
  label_master = [item.replace("\n", " ") for item in label_master]
  label_master = [item.replace("\r", " ") for item in label_master]
  label_master = [item.replace("  ", " ") for item in label_master]
BON_t = " ".join(label_master)

a=0
while a<co_len:
  connection[str(a)] = [conlist[a].attributes['from-id'].value,conlist[a].attributes['to-id'].value]
  if connection[str(a)][0] in concept:
    connection[str(a)][0] = concept[connection[str(a)][0]]
  elif connection[str(a)][0] in link:
    connection[str(a)][0] = link[connection[str(a)][0]]
  if connection[str(a)][1] in concept:
    connection[str(a)][1] = concept[connection[str(a)][1]]
  elif connection[str(a)][1] in link:
    connection[str(a)][1] = link[connection[str(a)][1]]
  connect1 = connection
  if(a>0):
    i=0
    while i<a:
      if(connection[str(a)][0] == connection[str(i)][0]):
        connection[str(i)].append(connection[str(a)][1])
        connection[str(a)][0] = " "
        connection[str(a)][1] = " "
      
      i=i+1
  a=a+1
a=0
i=0
while a<co_len:
  if(connection[str(a)][0]!=" "):
    final[i]=connection[str(a)]
    i=i+1
  a=a+1


a=0
i=0
while a<co_len:
  if(connection[str(a)][0] in link_master):
    j=0
    for i in connection[str(a)]:
      if(j==0):
        ans=i
        ans=ans.replace("\r","")
        
      else:
        final_f[ans].append(i)

      j=j+1
  a=a+1

print(final_f)


In [ ]:
print(label_master)
print(link_master)
linklis=[]
for ass in link_master:
  answe = ass.replace("\n"," ").replace("  "," ")
  linklis.append(answe)
print(linklis)

In [ ]:
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
from nltk import FreqDist
import pandas as pd
from csv import reader
 
def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'):
        return 'n'
 
    if tag.startswith('V'):
        return 'v'
 
    if tag.startswith('J'):
        return 'a'
 
    if tag.startswith('R'):
        return 'r'
 
    return None
 
def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
 
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None

def sentence_similarity(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet """
    # Tokenize and tag
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(word_tokenize(sentence2))
 
    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
    
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]

    score, count = 0.0, 0
 
    # For each word in the first sentence
    for synset in synsets1:
        simlist = [synset.path_similarity(ss) for ss in synsets2 if synset.path_similarity(ss) is not None]
        if not simlist:
            continue;
        best_score = max(simlist)
 
        # Check that the similarity could have been computed
        score += best_score
        count += 1

 
    # Average the values
    try:
      score /= count
    except Exception as e:
      score = 1.0
    return score
 
sentences = [

]

focus_sentence = rank_res
#link-node pair
for a in final_f:
  x=""
  x=a
  for b in final_f[a]:
    x = x+" "+b
  sentences.append(x)
CSVStudentAnswer = " ".join(sentences)
sum = 0
j=0
count = 0
res1=[]
for sentence in sentences:
    sen1 = sentence.lower()
    sim_res = sentence_similarity(sen1, focus_sentence)*100
    res1.append(sim_res)
    print ("Similarity(\"%s\") = %s" % (sentence, sim_res))
    

    if sim_res > 50:
      count = count+1
    j=j+1
sim_r=""
similarity_result= (count/j)*100 
if similarity_result>75:
  sim_r="Excellent"
elif similarity_result>50:
  sim_r="Good"
elif similarity_result>25:
  sim_r="Poor"
else:
  sim_r="Failing"

In [ ]:
a = {'Sentence': sentences, 'Wordnet similarity': res1}
df3 = pd.DataFrame.from_dict(a).to_csv(out_fi)

In [ ]:
def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'):
        return 'n'
 
    if tag.startswith('V'):
        return 'v'
 
    if tag.startswith('J'):
        return 'a'
 
    if tag.startswith('R'):
        return 'r'
 
    return None
 
def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
 
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None


def sentence_similarity(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet """
    # Tokenize and tag
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(word_tokenize(sentence2))
 
    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
    
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]

    score, count = 0.0, 0
 
    # For each word in the first sentence
    for synset in synsets1:
        simlist = [synset.path_similarity(ss) for ss in synsets2 if synset.path_similarity(ss) is not None]
        if not simlist:
            continue;
        best_score = max(simlist)
 
        # Check that the similarity could have been computed
        score += best_score
        count += 1

 
    # Average the values
    try:
      score /= count
    except Exception as e:
      score = 0.0
    return score
 
sentences = [

]

focus_sentence = rank_res
#link-node pair
for a in label_master:
  x=""
  x=a.replace("/"," or ").replace("  "," ")
  sentences.append(x)
CSVStudentAnswer = " ".join(sentences)
sum = 0
j=0
count = 0

res1=[]
for sentence in sentences:
    sen1 = sentence.lower()
    sim_res = sentence_similarity(sen1, focus_sentence)*100
    res1.append(sim_res)
    print ("Similarity(\"%s\") = %s" % (sentence, sim_res))
    if sim_res > 50:
      count = count+1
    j=j+1

bof_similarity_result= (count/j)*100
bof_similarity_result=round(bof_similarity_result)
print(bof_similarity_result)
bof_res=""
if bof_similarity_result>75:
  bof_res="Excellent"
elif bof_similarity_result>50:
  bof_res="Good"
elif bof_similarity_result>25:
  bof_res="Poor"
else:
  bof_res="Failing"


In [ ]:
a = {'Sentence': sentences, 'Wordnet similarity': res1}
df3 = pd.DataFrame.from_dict(a).to_csv(out_fil)

Embeddedness

In [ ]:
embed_res=(l_len/(c_len-1))*100

embed_res=round(embed_res)
print(embed_res)
embed_result =""
if embed_res>75:
  embed_result = "Excellent"
elif embed_res>50:
  embed_result = "Good"
elif embed_res>25:
  embed_result = "Poor"
else:
  embed_result = "Failing"

print(embed_result)

In [ ]:
res_link=[]
res_link1=[]
for i in link_master:
  link_res = i.replace("\n"," ").replace("\r"," ").replace("  "," ")
  
  res_link.append(link_res)

all_link = len(res_link)

uniq_link= len(Counter(res_link).keys())
efflink_result=""
efflink= (uniq_link/all_link)*100
efflink=round(efflink)
if efflink == 100:
  efflink_result="Excellent"
elif efflink > 75:
  efflink_result="Good"
else:
  cu = []
  su=0
  #can use this itself for efflink
  for a in final_f:
    cu.append(a)
  res_effi =[]
  for a in final_f:
    j=""
    cu.remove(a)
    j = " ".join(cu)
    resu = symmetric_sentence_similarity(a, j)*100
    res_effi.append(resu)
    cu.append(a)

  for n in res_effi:
    su = su + n

  link_sim=su/len(res_effi)
  if link_sim<50:
    efflink_result="Poor"
  else:
    efflink_result="Failing"

print(efflink_result)

In [ ]:
embed=collections.defaultdict(list)
for i in final:
  k=0
  for j in final[i]:
    if(k==0):
      ans=j
      ans=ans.replace("\r","").replace("  "," ")

    else:
      j=j.replace("\r","").replace("  "," ")
      embed[ans].append(j)

    k=k+1
print (embed)

In [ ]:
embed_link=collections.defaultdict(list)
links=[]
node_list=[]
for i in label_master:
  node_list.append(i)
for i in link_master:
  ans12=i.replace("\r"," ").replace("\n"," ").replace("  "," ")
  links.append(ans12)
for i in embed:
  if i in links:
    for j in embed[i]:
      embed_link[i].append(j)

for i in embed_link:
  for j in embed_link[i]:
    if j in node_list:
      node_list.remove(j)
    
print (node_list)

In [ ]:
print(final)
full=[]
for i in final:
  for j in final[i]:
    aswe=j.replace("  "," ")
    full.append(aswe)
print(full)
print(node_list)
for i in node_list:
  if i not in full:
    node_list.remove(i)


rootnode=node_list[0]
rootnode=rootnode.replace("  "," ")
print(node_list)

In [ ]:
print(rootnode)

#breadth if more than one rootnode

In [ ]:
res_D = []
if(len(node_list)>1):
  for i in node_list:
    rootnode = i
    depth=0
    dfs(visited, embed, rootnode, depth)
    max_dep = 0
    for i in depth_val:
      if i > max_dep:
        max_dep = i
    result_depth=""

    depth_res=(max_dep-1)/2
    depth_res=round(depth_res)
    res_D.append(depth_res)
    print(res_D)
    

In [ ]:
depth_res=0
for i in res_D:
  if(i>depth_res):
    depth_res = i
print(depth_res)
result_depth=sol_embed(depth_res)
print(result_depth)

#breadth if only one rootnode


In [ ]:
visited = set()
depth_val=[]
def dfs(visited, graph, node, depth):
    if node not in visited:
        visited.add(node)
        depth=depth+1
        depth_val.append(depth)
        for neighbour in graph[node]:
            dfs(visited, graph, neighbour, depth)

In [ ]:
depth=0

dfs(visited, embed, rootnode, depth)
print(depth_val)

In [ ]:
max_dep = 0
for i in depth_val:
  if i > max_dep:
    max_dep = i
result_depth=""

depth_res=(max_dep-1)/2
depth_res=round(depth_res)
if depth_res==0:
  result_depth = "Failing"
elif depth_res == 1:
  result_depth = "Poor"
elif depth_res == 2:
  result_depth = "Good"
else:
  result_depth = "Excellent"
print(result_depth)


In [ ]:
def check(check1):
  if check1 == "Excellent" :
    return 3
  elif check1 == "Good":
    return 2
  elif check1 == "Poor":
    return 1
  else:
    return 0

In [ ]:
def sol(val):
  if val == 3:
    return "Excellent"
  elif val == 2:
    return "Good"
  elif val == 1:
    return "Poor"
  else:
    return 0

In [ ]:
def sol_embed(val):
  if val > 3:
    return "Excellent"
  elif val == 2:
    return "Good"
  elif val == 1:
    return "Poor"
  else:
    return 0

In [ ]:
bn1=check(bof_res)

bn2=check(result_depth)

che=round((bn1+bn2)/2)
print(che)
bn3=sol(che)
print(bn3)

In [ ]:
efl1=check(efflink_result)
efl2=check(sim_r)
che2 = round((efl1+efl2)/2)

efl3=sol(che2)
print(efl3)

#Exporting output to csv

In [ ]:
import csv
import pandas as pd
from csv import reader

emb=[]
ef=[]
re=[]
bo=[]
si=[]
bof=[]
#Embed
emb.append(embed_result)
#efficient link
ef.append(efl3)
#descriptive link
si.append(sim_r)
#BON
re.append(result_depth)
bo.append(bof_res)
bof.append(bn3)
out_file = fil1.replace(".cxl","")
full_final = '/content/drive/My Drive/Auto_waterloo/res/try/' + out_file +'_AutoWaterloo.csv'
a = {'Breadth of net': bof,'Embeddedness': emb, "Use of descriptive links":si,'Efficient links': ef, 'Breadth of net_depth': depth_res, "Breadth of net_Nodes similarity ratio":bof_similarity_result, 'Embed value':embed_res, 'Use of descriptive link':similarity_result,'efficient_link val':efflink}
df3 = pd.DataFrame.from_dict(a).to_csv(full_final)
